# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [101]:
from azureml.core import Workspace, Experiment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, ComputeInstance, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment

from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import GridParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice 
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

from azureml.widgets import RunDetails

import joblib, os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [102]:
ws = Workspace.from_config()
experiment_name = 'og-azureml-capstone'

experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()

In [103]:
# Create compute cluster if there is no compute available 
cluster_name = list(ws.compute_targets.keys())[0]
if not(cluster_name):
    cluster_name = 'capstone-cluster'
    try:
        cluster = ComputeTarget(workspace = ws, name = cluster_name)
        print("Cluster already exists, start using it")
    except ComputeTargetException:
        compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", max_nodes = 4)
        cluster = ComputeTarget.create(ws, cluster_name, compute_config)

    cluster.wait_for_completion(show_output = True)
else:
    cluster = ComputeInstance(workspace = ws, name = cluster_name)
    print("Cluster already exists:", cluster_name)

Cluster already exists: notebook143014


In [104]:
# Upload dataset to the workspace
data_path = "https://raw.githubusercontent.com/olgagnatenko13/nd00333-capstone/master/dataset/Dataset_for_Classification.csv"
found = False
key = "attrition-analysis"
description_text = "Employee attrition data"

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    dataset = Dataset.Tabular.from_delimited_files(data_path)        
      
    dataset = dataset.register(workspace=ws, name=key, description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,...,3.153741,2.712245,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,...,0.360824,1.081209,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,3.000000,2.000000,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,...,3.000000,3.000000,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,...,3.000000,4.000000,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,4.000000,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [105]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling
# BanditPolicy allows to set up a relatively "aggressive" termination approach, and therefore allows to save VM resources
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1, delay_evaluation = 5)

#TODO: Create the different params that you will be using during training
# Grid sampling is used since both hyperparameters are selected from a finite list of values,
# and each value has its own impact on model performance
param_sampling = GridParameterSampling( {
    # "--C": choice(0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000),
    "--C": choice(0.01, 0.05, 0.1, 0.5),
    "--gamma": choice(1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001)
    }
)

In [106]:
# prepare folder for training data 
source_directory = "./training"
if "training" not in os.listdir():
    os.mkdir(source_directory)

In [113]:
#TODO: Create your estimator and hyperdrive config
# since SKLearn estimator is deprecated, I use ScriptRunConfig

environment = Environment.get(workspace = ws, name = "AzureML-Tutorial")

script_run_config = ScriptRunConfig(
    source_directory=source_directory,
    script = "train.py",
    compute_target = cluster,
    environment = environment
)

hyperdrive_run_config = HyperDriveConfig(
	run_config = script_run_config,
	hyperparameter_sampling = param_sampling,
	policy = early_termination_policy,
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 2, # 30,
    max_duration_minutes = 60
)

In [114]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [115]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_890e12bb-b578-448e-bd44-7f1503aada9e
Web View: https://ml.azure.com/runs/HD_890e12bb-b578-448e-bd44-7f1503aada9e?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-143014/workspaces/quick-starts-ws-143014&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-18T11:48:10.849415][API][INFO]Experiment created<END>\n""<START>[2021-04-18T11:48:11.931244][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n"<START>[2021-04-18T11:48:12.2093182Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-04-18T11:48:12.495682][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_890e12bb-b578-448e-bd44-7f1503aada9e
Web View: https://ml.azure.com/runs/HD_890e12bb-b578-448e-bd44-7f1503aada9e?wsid=/subscriptions/a0a76bad-11a1-4a2d-9

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User errors were found in at least one of the child runs.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print("\nBest run metrics:", best_run_metrics)
print("\nParameter values:", parameter_values)

files = best_run.get_file_names()
best_model_name = files[-1]
best_run.download_file(best_model_name, output_file_path = "./outputs/")

best_model = joblib.load(best_model_name)
print("BEST MODEL:", best_model_name, best_model)

In [ ]:
#TODO: Save the best model
output_folder='./outputs'
os.makedirs(output_folder, exist_ok=True)

joblib.dump(value=best_model, filename=best_model_name)

best_model = best_run.register_model(model_name = "best_hd_model", model_path = best_model_name, tags = best_run_metrics)
hd_accuracy = best_run_metrics["Accuracy"]
print("Accuracy: ", hd_accuracy)
print("Best run metrics: ", best_run_metrics)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service